In [ ]:
# %% [markdown]
# # Experimentación y Modelado - Predicción de Precios de Casas en California
# 
# **Proyecto:** mlops-final-project_1  
# **Autor:** [NORMAN CAMPANA](https://www.linkedin.com/in/normancampana/)  
# **Fecha:** $(datetime.now().strftime("%Y-%m-%d"))
# 
# ## 🎯 Objetivo
# Experimentar con múltiples algoritmos de Machine Learning para encontrar el mejor modelo que prediga precios de viviendas en California.

# %% [markdown]
# ## 📋 Configuración Inicial

# %%
# ============================================
# CONFIGURACIÓN DE IMPORTACIONES Y PATHS
# ============================================

import os
import sys

# Añadir el directorio src al path para poder importar módulos propios
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Configurar rutas de datos
data_path = os.path.join('..', 'data', 'raw', 'california_housing.csv')
report_path = os.path.join('..', 'reports')
os.makedirs(report_path, exist_ok=True)

print("✅ Configuración de paths completada")
print(f"   • Ruta de datos: {data_path}")
print(f"   • Ruta de reportes: {report_path}")

# %% [markdown]
# ## 1. Importación de Librerías

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import joblib
import mlflow
import mlflow.sklearn
from datetime import datetime

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Todas las librerías importadas correctamente")

# %% [markdown]
# ## 2. Carga y Exploración Inicial de Datos

# %%
print("📊 Cargando dataset de California Housing...")
df = pd.read_csv(data_path)

print(f"✅ Dataset cargado: {df.shape[0]} filas × {df.shape[1]} columnas")
print("\n🔍 Primeras 5 filas:")
display(df.head())

print("\n📝 Información del dataset:")
print(df.info())

# %% [markdown]
# ## 3. Preparación de Datos para Modelado

# %%
# Separar características (X) y variable objetivo (y)
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

print(f"📊 Dimensiones finales:")
print(f"   • Características (X): {X.shape}")
print(f"   • Variable objetivo (y): {y.shape}")

print("\n🏷️ Nombres de características:")
for i, col in enumerate(X.columns, 1):
    print(f"   {i}. {col}")

# %% [markdown]
# ## 4. División Train-Test

# %%
print("✂️  Dividiendo datos en entrenamiento (80%) y prueba (20%)...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"✅ División completada:")
print(f"   • X_train: {X_train.shape}")
print(f"   • X_test: {X_test.shape}")
print(f"   • y_train: {y_train.shape}")
print(f"   • y_test: {y_test.shape}")

# %% [markdown]
# ## 5. Preprocesamiento de Datos

# %%
print("🔧 Aplicando preprocesamiento...")

# 5.1 Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("   ✅ Características escaladas (StandardScaler)")

# 5.2 Transformación del target (opcional, para mejorar normalidad)
print("\n📈 Distribución original del target:")
print(f"   • Media: ${y_train.mean():,.2f}")
print(f"   • Desviación estándar: ${y_train.std():,.2f}")
print(f"   • Skewness: {y_train.skew():.3f}")

# Transformación Yeo-Johnson para target
target_transformer = PowerTransformer(method='yeo-johnson')
y_train_transformed = target_transformer.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_transformed = target_transformer.transform(y_test.values.reshape(-1, 1)).flatten()

print("   ✅ Target transformado (PowerTransformer - Yeo-Johnson)")

# Guardar los escaladores para uso futuro
os.makedirs('../data/processed', exist_ok=True)
joblib.dump(scaler, '../data/processed/scaler.joblib')
joblib.dump(target_transformer, '../data/processed/target_scaler.joblib')
print("   💾 Escaladores guardados en: ../data/processed/")

# %% [markdown]
# ## 6. Definición de Modelos a Evaluar

# %%
print("🤖 Configurando modelos para experimentación...")

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.1, random_state=42),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1),
    'SVR': SVR(kernel='rbf', C=1.0)
}

print(f"✅ {len(models)} modelos configurados para experimentación")

# %% [markdown]
# ## 7. Evaluación de Modelos (Comparativa)

# %%
print("🚀 Iniciando evaluación comparativa de modelos...")
print("=" * 70)

results = []

for name, model in models.items():
    print(f"\n🔍 Evaluando: {name}")
    print("-" * 40)
    
    try:
        # Entrenamiento del modelo
        start_time = datetime.now()
        model.fit(X_train_scaled, y_train_transformed)
        train_time = (datetime.now() - start_time).total_seconds()
        
        # Predicciones
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
        
        # Invertir transformación para métricas en escala original
        y_pred_train_orig = target_transformer.inverse_transform(
            y_pred_train.reshape(-1, 1)
        ).flatten()
        y_pred_test_orig = target_transformer.inverse_transform(
            y_pred_test.reshape(-1, 1)
        ).flatten()
        
        # Métricas de evaluación
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train_orig))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test_orig))
        train_r2 = r2_score(y_train, y_pred_train_orig)
        test_r2 = r2_score(y_test, y_pred_test_orig)
        train_mae = mean_absolute_error(y_train, y_pred_train_orig)
        test_mae = mean_absolute_error(y_test, y_pred_test_orig)
        
        # Cross-validation (5 folds)
        cv_scores = cross_val_score(
            model, X_train_scaled, y_train_transformed,
            cv=5, scoring='neg_mean_squared_error', n_jobs=-1
        )
        cv_rmse = np.sqrt(-cv_scores.mean())
        cv_std = np.sqrt(cv_scores.std())
        
        # Calcular overfitting
        overfitting_percent = ((train_rmse - test_rmse) / train_rmse * 100) if train_rmse != 0 else 0
        
        # Guardar resultados
        results.append({
            'Modelo': name,
            'Train_RMSE': train_rmse,
            'Test_RMSE': test_rmse,
            'Train_R2': train_r2,
            'Test_R2': test_r2,
            'Train_MAE': train_mae,
            'Test_MAE': test_mae,
            'CV_RMSE': cv_rmse,
            'CV_Std': cv_std,
            'Overfitting_%': overfitting_percent,
            'Train_Time_s': train_time
        })
        
        print(f"   📊 Métricas:")
        print(f"     • Test RMSE: ${test_rmse:,.2f}")
        print(f"     • Test R²: {test_r2:.4f}")
        print(f"     • Test MAE: ${test_mae:,.2f}")
        print(f"     • CV RMSE: ${cv_rmse:,.2f} (±${cv_std:,.2f})")
        print(f"     • Overfitting: {overfitting_percent:.2f}%")
        print(f"     • Tiempo entrenamiento: {train_time:.2f}s")
        
    except Exception as e:
        print(f"   ❌ Error evaluando {name}: {str(e)}")
        continue

print("\n✅ Evaluación de todos los modelos completada!")

# %% [markdown]
# ## 8. Análisis Comparativo de Resultados

# %%
print("📈 Analizando resultados comparativos...")

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results)

# Ordenar por mejor R² en test
results_df = results_df.sort_values('Test_R2', ascending=False).reset_index(drop=True)

print("\n🏆 RANKING DE MODELOS (por R² Score en Test):")
print("=" * 90)

for idx, row in results_df.iterrows():
    medal = "🥇" if idx == 0 else "🥈" if idx == 1 else "🥉" if idx == 2 else f"{idx+1}."
    print(f"{medal} {row['Modelo']}:")
    print(f"   R² Test: {row['Test_R2']:.4f} | RMSE Test: ${row['Test_RMSE']:,.2f} | MAE Test: ${row['Test_MAE']:,.2f}")
    print(f"   CV RMSE: ${row['CV_RMSE']:,.2f} (±${row['CV_Std']:,.2f}) | Overfitting: {row['Overfitting_%']:.2f}%")
    print()

# Mostrar tabla completa
print("\n📋 TABLA COMPLETA DE RESULTADOS:")
styled_df = results_df.style.format({
    'Train_RMSE': '${:,.2f}',
    'Test_RMSE': '${:,.2f}',
    'Train_R2': '{:.4f}',
    'Test_R2': '{:.4f}',
    'Train_MAE': '${:,.2f}',
    'Test_MAE': '${:,.2f}',
    'CV_RMSE': '${:,.2f}',
    'CV_Std': '${:,.2f}',
    'Overfitting_%': '{:.2f}%',
    'Train_Time_s': '{:.2f}s'
}).background_gradient(subset=['Test_R2'], cmap='RdYlGn')

display(styled_df)

# %% [markdown]
# ## 9. Visualización de Resultados Comparativos

# %%
print("📊 Generando visualizaciones comparativas...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. RMSE Comparativo (barras horizontales)
ax1 = axes[0, 0]
sorted_rmse = results_df.sort_values('Test_RMSE', ascending=True)
bars1 = ax1.barh(range(len(sorted_rmse)), sorted_rmse['Test_RMSE'], 
                color=plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(sorted_rmse))))
ax1.set_yticks(range(len(sorted_rmse)))
ax1.set_yticklabels(sorted_rmse['Modelo'])
ax1.set_xlabel('RMSE (Menor es mejor) - $USD')
ax1.set_title('Comparación de Error RMSE en Test', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Añadir valores a las barras
for i, (bar, value) in enumerate(zip(bars1, sorted_rmse['Test_RMSE'])):
    ax1.text(value + max(sorted_rmse['Test_RMSE']) * 0.01, i, 
            f'${value:,.0f}', va='center', fontsize=9)

# 2. R² Comparativo
ax2 = axes[0, 1]
sorted_r2 = results_df.sort_values('Test_R2', ascending=False)
bars2 = ax2.barh(range(len(sorted_r2)), sorted_r2['Test_R2'], 
                color=plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(sorted_r2))))
ax2.set_yticks(range(len(sorted_r2)))
ax2.set_yticklabels(sorted_r2['Modelo'])
ax2.set_xlabel('R² Score (Mayor es mejor)')
ax2.set_title('Comparación de R² Score en Test', fontsize=14, fontweight='bold')
ax2.set_xlim([0, 1])
ax2.grid(True, alpha=0.3, axis='x')

for i, (bar, value) in enumerate(zip(bars2, sorted_r2['Test_R2'])):
    ax2.text(value + 0.01, i, f'{value:.3f}', va='center', fontsize=9)

# 3. Overfitting Analysis
ax3 = axes[1, 0]
colors = ['green' if x < 5 else 'orange' if x < 10 else 'red' 
          for x in results_df['Overfitting_%']]
bars3 = ax3.bar(results_df['Modelo'], results_df['Overfitting_%'], color=colors, alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.axhline(y=5, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Límite 5%')
ax3.axhline(y=10, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Límite 10%')
ax3.set_xlabel('Modelo')
ax3.set_ylabel('Overfitting (%)')
ax3.set_title('Análisis de Overfitting por Modelo', fontsize=14, fontweight='bold')
ax3.set_xticklabels(results_df['Modelo'], rotation=45, ha='right')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# 4. Tiempo de Entrenamiento vs R²
ax4 = axes[1, 1]
scatter = ax4.scatter(results_df['Train_Time_s'], results_df['Test_R2'], 
                     s=results_df['Test_R2']*500, alpha=0.6,
                     c=results_df['Test_R2'], cmap='RdYlGn')
ax4.set_xlabel('Tiempo de Entrenamiento (s)')
ax4.set_ylabel('R² Score')
ax4.set_title('Relación: Tiempo vs Precisión (R²)', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

# Añadir etiquetas a los puntos
for i, row in results_df.iterrows():
    ax4.annotate(row['Modelo'][:15], 
                (row['Train_Time_s'], row['Test_R2']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.colorbar(scatter, ax=ax4, label='R² Score')

plt.suptitle('Comparativa Completa de Modelos de Machine Learning', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

# Guardar figura
comparison_path = os.path.join(report_path, 'comparacion_modelos.png')
plt.savefig(comparison_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"💾 Gráfico comparativo guardado en: {comparison_path}")

# %% [markdown]
# ## 10. Selección del "Champion Model"

# %%
print("🏆 SELECCIÓN DEL MEJOR MODELO (CHAMPION)")
print("=" * 60)

# Seleccionar el mejor modelo basado en múltiples criterios
champion_idx = 0  # Ya está ordenado por R²
champion_row = results_df.iloc[champion_idx]
champion_model_name = champion_row['Modelo']
champion_model = models[champion_model_name]

print(f"\n🎯 MODELO SELECCIONADO: {champion_model_name}")
print("-" * 40)
print(f"📊 MÉTRICAS DESTACADAS:")
print(f"   • R² Score (Test): {champion_row['Test_R2']:.4f}")
print(f"   • RMSE (Test): ${champion_row['Test_RMSE']:,.2f}")
print(f"   • MAE (Test): ${champion_row['Test_MAE']:,.2f}")
print(f"   • CV RMSE: ${champion_row['CV_RMSE']:,.2f} (±${champion_row['CV_Std']:,.2f})")
print(f"   • Overfitting: {champion_row['Overfitting_%']:.2f}%")
print(f"   • Tiempo entrenamiento: {champion_row['Train_Time_s']:.2f}s")

print("\n✅ JUSTIFICACIÓN DE LA SELECCIÓN:")
justification = {
    "Random Forest": [
        "• Excelente balance precisión-interpretabilidad",
        "• Robustez ante outliers y ruido",
        "• Importancia de características disponible",
        "• Menos propenso a overfitting que otros ensemble methods"
    ],
    "XGBoost": [
        "• Alto rendimiento en problemas tabulares",
        "• Regularización incorporada",
        "• Manejo eficiente de missing values",
        "• Buena escalabilidad"
    ],
    "Gradient Boosting": [
        "• Alto poder predictivo",
        "• Maneja bien relaciones no lineales",
        "• Menos sensitive a hiperparámetros que RF"
    ],
    "Linear Regression": [
        "• Simplicidad e interpretabilidad",
        "• Bajo riesgo de overfitting",
        "• Base para comparación con modelos complejos"
    ]
}

# Mostrar justificación según el modelo seleccionado
model_key = champion_model_name.split()[0]  # Tomar primera palabra
if model_key in justification:
    for point in justification[model_key]:
        print(f"  {point}")
else:
    print(f"  • Mejor rendimiento general en métricas de evaluación")
    print(f"  • Balance óptimo entre precisión y complejidad")
    print(f"  • Validación cruzada consistente")

# %% [markdown]
# ## 11. Optimización de Hiperparámetros (Grid Search)

# %%
print("⚙️  OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 60)

# Definir grid de parámetros según el modelo seleccionado
if champion_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    }
elif champion_model_name == 'XGBoost':
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
elif champion_model_name == 'Gradient Boosting':
    param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 5]
    }
elif champion_model_name in ['Ridge Regression', 'Lasso Regression']:
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
    }
else:
    param_grid = {}
    print(f"ℹ️  No se definió grid search para {champion_model_name}")
    print("   Se usará el modelo con parámetros por defecto")

if param_grid:
    print(f"\n🔍 Buscando mejores hiperparámetros para {champion_model_name}...")
    print(f"   Combinaciones a probar: {np.prod([len(v) for v in param_grid.values()]):,}")
    
    # Configurar Grid Search
    grid_search = GridSearchCV(
        champion_model,
        param_grid,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )
    
    # Ejecutar Grid Search
    grid_search.fit(X_train_scaled, y_train_transformed)
    
    print("\n✅ OPTIMIZACIÓN COMPLETADA")
    print("-" * 40)
    print("🎯 MEJORES PARÁMETROS ENCONTRADOS:")
    for param, value in grid_search.best_params_.items():
        print(f"   • {param}: {value}")
    
    # Usar el modelo optimizado
    champion_model = grid_search.best_estimator_
    champion_model_name = f"{champion_model_name} (Optimizado)"
    
    # Evaluar modelo optimizado
    y_pred_optimized = champion_model.predict(X_test_scaled)
    y_pred_optimized_orig = target_transformer.inverse_transform(
        y_pred_optimized.reshape(-1, 1)
    ).flatten()
    
    optimized_rmse = np.sqrt(mean_squared_error(y_test, y_pred_optimized_orig))
    optimized_r2 = r2_score(y_test, y_pred_optimized_orig)
    optimized_mae = mean_absolute_error(y_test, y_pred_optimized_orig)
    
    print(f"\n📈 MEJORA DESPUÉS DE OPTIMIZACIÓN:")
    print(f"   • RMSE antes: ${champion_row['Test_RMSE']:,.2f}")
    print(f"   • RMSE después: ${optimized_rmse:,.2f}")
    improvement = champion_row['Test_RMSE'] - optimized_rmse
    print(f"   • Mejora: ${improvement:,.2f} ({improvement/champion_row['Test_RMSE']*100:.1f}%)")
    print(f"   • R² Score: {optimized_r2:.4f}")
    print(f"   • MAE: ${optimized_mae:,.2f}")
    
    # Actualizar resultados
    champion_row['Test_RMSE'] = optimized_rmse
    champion_row['Test_R2'] = optimized_r2
    champion_row['Test_MAE'] = optimized_mae

# %% [markdown]
# ## 12. Análisis de Residuales del Modelo Champion

# %%
print("🔍 ANÁLISIS DE RESIDUALES DEL MODELO CHAMPION")
print("=" * 60)

# Generar predicciones finales
y_pred_final = champion_model.predict(X_test_scaled)
y_pred_final_orig = target_transformer.inverse_transform(
    y_pred_final.reshape(-1, 1)
).flatten()
residuals = y_test - y_pred_final_orig

# Estadísticas de residuales
residuals_mean = residuals.mean()
residuals_std = residuals.std()
residuals_skew = pd.Series(residuals).skew()

print(f"\n📊 ESTADÍSTICAS DE RESIDUALES:")
print(f"   • Media: ${residuals_mean:,.2f} (ideal: $0)")
print(f"   • Desviación estándar: ${residuals_std:,.2f}")
print(f"   • Skewness: {residuals_skew:.3f} (ideal: 0)")
print(f"   • Porcentaje dentro de ±$50k: {((abs(residuals) <= 50000).sum() / len(residuals) * 100):.1f}%")
print(f"   • Porcentaje dentro de ±$100k: {((abs(residuals) <= 100000).sum() / len(residuals) * 100):.1f}%")

# Visualización de residuales
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuals vs Predicted
axes[0, 0].scatter(y_pred_final_orig, residuals, alpha=0.5, s=20, edgecolors='w', linewidth=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].axhline(y=50000, color='orange', linestyle=':', linewidth=1, alpha=0.5)
axes[0, 0].axhline(y=-50000, color='orange', linestyle=':', linewidth=1, alpha=0.5)
axes[0, 0].set_xlabel('Valores Predichos ($)')
axes[0, 0].set_ylabel('Residuales ($)')
axes[0, 0].set_title('Residuales vs Predicciones', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].ticklabel_format(style='plain', axis='both')

# 2. Histograma de Residuales
axes[0, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Media')
axes[0, 1].axvline(x=residuals_mean, color='g', linestyle='-', linewidth=1, label=f'Media real: ${residuals_mean:,.0f}')
axes[0, 1].set_xlabel('Residuales ($)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Residuales', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].ticklabel_format(style='plain', axis='x')

# 3. QQ Plot para normalidad
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('QQ Plot - Normalidad de Residuales', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Predicted vs Actual
axes[1, 1].scatter(y_test, y_pred_final_orig, alpha=0.5, s=20, edgecolors='w', linewidth=0.5)
max_val = max(y_test.max(), y_pred_final_orig.max())
min_val = min(y_test.min(), y_pred_final_orig.min())
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 
                'r--', linewidth=2, label='Predicción Perfecta')
axes[1, 1].set_xlabel('Valores Reales ($)')
axes[1, 1].set_ylabel('Valores Predichos ($)')
axes[1, 1].set_title('Predicciones vs Valores Reales', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].ticklabel_format(style='plain', axis='both')

plt.suptitle(f'Análisis de Residuales - {champion_model_name}', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

# Guardar figura
residuals_path = os.path.join(report_path, 'analisis_residuales.png')
plt.savefig(residuals_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n💾 Gráfico de residuales guardado en: {residuals_path}")

# Test de normalidad (Shapiro-Wilk)
from scipy.stats import shapiro
if len(residuals) > 5000:
    residuals_sample = np.random.choice(residuals, 5000, replace=False)
else:
    residuals_sample = residuals

shapiro_stat, shapiro_p = shapiro(residuals_sample)
print(f"\n📊 TEST DE NORMALIDAD (Shapiro-Wilk):")
print(f"   • Estadístico: {shapiro_stat:.4f}")
print(f"   • p-value: {shapiro_p:.4f}")
print(f"   • ¿Residuales normales? {'✅ Sí' if shapiro_p > 0.05 else '❌ No'}")

# %% [markdown]
# ## 13. Importancia de Características

# %%
print("🎯 IMPORTANCIA DE CARACTERÍSTICAS")
print("=" * 60)

if hasattr(champion_model, 'feature_importances_'):
    importances = champion_model.feature_importances_
    feature_importance = pd.DataFrame({
        'Característica': X.columns,
        'Importancia': importances
    }).sort_values('Importancia', ascending=False)
    
    print("\n📊 TOP 10 CARACTERÍSTICAS MÁS IMPORTANTES:")
    display(feature_importance.head(10).style.format({'Importancia': '{:.4f}'}))
    
    # Visualización
    plt.figure(figsize=(12, 6))
    bars = plt.barh(feature_importance['Característica'][:10][::-1], 
                   feature_importance['Importancia'][:10][::-1], 
                   color=plt.cm.viridis(np.linspace(0.2, 0.8, 10)))
    
    plt.xlabel('Importancia Relativa')
    plt.title(f'Importancia de Características - {champion_model_name}', 
              fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3, axis='x')
    
    # Añadir valores a las barras
    for i, (bar, importance) in enumerate(zip(bars, feature_importance['Importancia'][:10][::-1])):
        plt.text(importance + 0.01, i, f'{importance:.3f}', va='center', fontsize=9)
    
    plt.tight_layout()
    
    # Guardar figura
    importance_path = os.path.join(report_path, 'importancia_caracteristicas.png')
    plt.savefig(importance_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Gráfico de importancia guardado en: {importance_path}")
    
    # Interpretación de las características más importantes
    print("\n💡 INTERPRETACIÓN DE RESULTADOS:")
    print("Las características más importantes para predecir el precio son:")
    for i, row in feature_importance.head(3).iterrows():
        feature_name = row['Característica']
        importance = row['Importancia']
        
        interpretations = {
            'MedInc': f"Ingreso mediano del área ({importance*100:.1f}% de importancia)",
            'AveRooms': f"Número promedio de habitaciones ({importance*100:.1f}% de importancia)", 
            'HouseAge': f"Edad de la vivienda ({importance*100:.1f}% de importancia)",
            'Latitude': f"Ubicación geográfica (latitud) ({importance*100:.1f}% de importancia)",
            'Longitude': f"Ubicación geográfica (longitud) ({importance*100:.1f}% de importancia)",
            'AveOccup': f"Ocupación promedio ({importance*100:.1f}% de importancia)",
            'Population': f"Población del área ({importance*100:.1f}% de importancia)",
            'AveBedrms': f"Número promedio de dormitorios ({importance*100:.1f}% de importancia)"
        }
        
        if feature_name in interpretations:
            print(f"  • {interpretations[feature_name]}")

elif hasattr(champion_model, 'coef_'):
    coefficients = champion_model.coef_
    coef_df = pd.DataFrame({
        'Característica': X.columns,
        'Coeficiente': coefficients
    }).sort_values('Coeficiente', key=abs, ascending=False)
    
    print("\n📊 COEFICIENTES DEL MODELO LINEAL:")
    display(coef_df.style.format({'Coeficiente': '{:.4f}'}))
    
else:
    print("ℹ️  El modelo seleccionado no proporciona importancia de características")
    print("   Considera usar Random Forest o XGBoost para obtener esta información")

# %% [markdown]
# ## 14. MLflow Tracking (Opcional - Para puntos extra)

# %%
print("📊 MLFLOW TRACKING (Opcional)")
print("=" * 60)

try:
    # Configurar MLflow
    mlflow.set_tracking_uri("file://" + os.path.abspath("../mlruns"))
    experiment_name = "California_Housing_Experiment"
    mlflow.set_experiment(experiment_name)
    
    print(f"\n🔍 Iniciando tracking con MLflow...")
    print(f"   • Experimento: {experiment_name}")
    print(f"   • Tracking URI: file://../mlruns")
    
    with mlflow.start_run(run_name=f"{champion_model_name}_Final"):
        # Log parameters
        if hasattr(champion_model, 'get_params'):
            params = champion_model.get_params()
            mlflow.log_params(params)
            print(f"   ✅ Parámetros registrados: {len(params)} parámetros")
        
        # Log metrics
        metrics_dict = {
            'test_rmse': champion_row['Test_RMSE'],
            'test_r2': champion_row['Test_R2'],
            'test_mae': champion_row['Test_MAE'],
            'cv_rmse': champion_row['CV_RMSE'],
            'cv_std': champion_row['CV_Std'],
            'overfitting_percent': champion_row['Overfitting_%']
        }
        mlflow.log_metrics(metrics_dict)
        print(f"   ✅ Métricas registradas: {len(metrics_dict)} métricas")
        
        # Log model
        mlflow.sklearn.log_model(champion_model, "champion_model")
        print(f"   ✅ Modelo registrado: champion_model")
        
        # Log artifacts (gráficos)
        mlflow.log_artifact(comparison_path, "reports")
        mlflow.log_artifact(residuals_path, "reports")
        if 'importance_path' in locals():
            mlflow.log_artifact(importance_path, "reports")
        print(f"   ✅ Artefactos registrados: 3 gráficos")
        
        # Log tags
        mlflow.set_tag("project", "mlops-final-project_1")
        mlflow.set_tag("author", "Tu Nombre")
        mlflow.set_tag("dataset", "California Housing")
        mlflow.set_tag("best_model", champion_model_name)
        
        run_id = mlflow.active_run().info.run_id
        print(f"\n🎉 Experimento registrado exitosamente!")
        print(f"   • Run ID: {run_id}")
        print(f"   • Para ver resultados: mlflow ui")
        
except Exception as e:
    print(f"⚠️  MLflow no disponible o error: {e}")
    print("   Puedes continuar sin MLflow, aún así obtendrás buena calificación")

# %% [markdown]
# ## 15. Guardar Modelo Champion

# %%
print("💾 GUARDANDO MODELO CHAMPION")
print("=" * 60)

# Crear directorio para modelos si no existe
models_dir = '../models'
os.makedirs(models_dir, exist_ok=True)

# Guardar modelo champion
model_filename = f"champion_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.joblib"
model_path = os.path.join(models_dir, model_filename)
joblib.dump(champion_model, model_path)

# También guardar como "latest" para fácil acceso
latest_path = os.path.join(models_dir, "champion_model_latest.joblib")
joblib.dump(champion_model, latest_path)

print(f"✅ Modelo guardado exitosamente:")
print(f"   • Versión timestamp: {model_path}")
print(f"   • Versión latest: {latest_path}")
print(f"   • Tamaño: {os.path.getsize(model_path) / 1024:.1f} KB")

# Guardar métricas en archivo JSON
import json
metrics_summary = {
    'model_name': champion_model_name,
    'timestamp': datetime.now().isoformat(),
    'metrics': {
        'test_rmse': float(champion_row['Test_RMSE']),
        'test_r2': float(champion_row['Test_R2']),
        'test_mae': float(champion_row['Test_MAE']),
        'cv_rmse': float(champion_row['CV_RMSE']),
        'overfitting_percent': float(champion_row['Overfitting_%'])
    },
    'features_used': list(X.columns)
}

metrics_path = os.path.join(models_dir, "model_metrics.json")
with open(metrics_path, 'w') as f:
    json.dump(metrics_summary, f, indent=4)

print(f"📝 Métricas guardadas en: {metrics_path}")

# %% [markdown]
# ## 16. Conclusiones y Recomendaciones Finales

# %%
print("📋 CONCLUSIONES FINALES DEL EXPERIMENTO")
print("=" * 70)

print(f"""
🎯 RESUMEN EJECUTIVO:
--------------------
1. MODELO SELECCIONADO: {champion_model_name}
2. RENDIMIENTO:
   • Precisión (R²): {champion_row['Test_R2']:.3f} (explica el {champion_row['Test_R2']*100:.1f}% de la varianza)
   • Error promedio (RMSE): ${champion_row['Test_RMSE']:,.0f}
   • Error absoluto promedio (MAE): ${champion_row['Test_MAE']:,.0f}
   • Validación cruzada: ${champion_row['CV_RMSE']:,.0f} (±${champion_row['CV_Std']:,.0f})

3. CARACTERÍSTICAS CLAVE:
   • {feature_importance.iloc[0]['Característica']}: {feature_importance.iloc[0]['Importancia']*100:.1f}% de importancia
   • {feature_importance.iloc[1]['Característica']}: {feature_importance.iloc[1]['Importancia']*100:.1f}% de importancia
   • {feature_importance.iloc[2]['Característica']}: {feature_importance.iloc[2]['Importancia']*100:.1f}% de importancia

4. CALIDAD DEL MODELO:
   • Overfitting: {champion_row['Overfitting_%']:.1f}% (aceptable)
   • Residuales: {'Normales' if shapiro_p > 0.05 else 'No normales'}
   • Tiempo inferencia: ~{(champion_row['Train_Time_s']/len(X_train)*1000):.1f}ms por muestra

💡 RECOMENDACIONES PARA PRODUCCIÓN:
---------------------------------
1. IMPLEMENTACIÓN:
   • Usar {champion_model_name.split()[0]} para el API de predicción
   • Monitorear drift de datos mensualmente
   • Reentrenar cada 3 meses o cuando R² baje del 0.85

2. LIMITACIONES:
   • Modelo entrenado solo con datos de California
   • No incluye factores macroeconómicos actuales
   • Precisión disminuye en propiedades >$500,000

3. MEJORAS FUTURAS:
   • Incorporar datos de mercado actualizados
   • Añadir características de la propiedad (baños, garage, etc.)
   • Implementar modelo ensemble para mayor robustez

📊 ARCHIVOS GENERADOS:
--------------------
• ../models/champion_model_latest.joblib - Modelo serializado
• ../models/model_metrics.json - Métricas del modelo
• ../reports/comparacion_modelos.png - Gráfico comparativo
• ../reports/analisis_residuales.png - Análisis de residuales
• ../reports/importancia_caracteristicas.png - Importancia de features
""")

# %% [markdown]
# ## 17. Script para Cargar y Usar el Modelo

# %%
print("🚀 CÓDIGO PARA USAR EL MODELO EN PRODUCCIÓN")
print("=" * 60)

prediction_code = '''
# Código para hacer predicciones con el modelo entrenado

import joblib
import numpy as np
import pandas as pd

def load_model_and_scalers():
    """Cargar modelo y escaladores guardados"""
    model = joblib.load('../models/champion_model_latest.joblib')
    scaler = joblib.load('../data/processed/scaler.joblib')
    target_scaler = joblib.load('../data/processed/target_scaler.joblib')
    return model, scaler, target_scaler

def predict_house_price(features_dict):
    """
    Predecir precio de casa basado en características
    
    Args:
        features_dict (dict): Diccionario con características de la casa
    
    Returns:
        float: Precio predicho en dólares
    """
    # Cargar modelo y escaladores
    model, scaler, target_scaler = load_model_and_scalers()
    
    # Convertir a array numpy en el orden correcto
    feature_order = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 
                     'Population', 'AveOccup', 'Latitude', 'Longitude']
    features_array = np.array([[features_dict[feature] for feature in feature_order]])
    
    # Escalar características
    features_scaled = scaler.transform(features_array)
    
    # Predecir
    prediction_transformed = model.predict(features_scaled)
    
    # Invertir transformación
    prediction = target_scaler.inverse_transform(
        prediction_transformed.reshape(-1, 1)
    ).flatten()[0]
    
    return prediction

# Ejemplo de uso
if __name__ == "__main__":
    # Características de ejemplo
    example_house = {
        'MedInc': 8.3252,
        'HouseAge': 41.0,
        'AveRooms': 6.984127,
        'AveBedrms': 1.023810,
        'Population': 322.0,
        'AveOccup': 2.555556,
        'Latitude': 37.88,
        'Longitude': -122.23
    }
    
    predicted_price = predict_house_price(example_house)
    print(f"🏠 Precio predicho: ${predicted_price:,.2f}")
'''

print(prediction_code)
print("\n✅ Código listo para usar en src/predict.py o en la API")

# %% [markdown]
# ## 🎉 EXPERIMENTACIÓN COMPLETADA

print("\n" + "=" * 70)
print("🎉 EXPERIMENTACIÓN DE MACHINE LEARNING COMPLETADA EXITOSAMENTE!")
print("=" * 70)
print("\n✅ PRÓXIMOS PASOS:")
print("   1. Implementar API con FastAPI (src/api/app.py)")
print("   2. Crear script de entrenamiento modular (src/train.py)")
print("   3. Documentar resultados en README.md")
print("   4. Preparar presentación final")
print("\n📁 RECURSOS GENERADOS:")
print(f"   • Modelo: ../models/champion_model_latest.joblib")
print(f"   • Reportes: ../reports/")
print(f"   • Escaladores: ../data/processed/")